<img src="https://github.com/retkowsky/visual-search-azureAI/blob/main/logo.jpg?raw=true">

# Fashion Visual Search demo

## 4. Visual search webapp

This code demonstrates how to use **Azure Cognitive Search** with **Cognitive Services Florence Vision API** and Azure Python SDK for visual search.


## Steps
- Connect to a blob storage where your catalog images are
- Use Azure Computer Vision 4 to embed all these images
- Create an Azure Cognitive search vector store index
- Upload the embeddings into an Azure Cognitive Search index
- Do some visual search using a prompt or an image


## Visual search with vector embeddings
Vector embeddings are a way of representing content such as text or images as vectors of real numbers in a high-dimensional space. These embeddings are often learned from large amounts of textual and visual data using machine learning algorithms like neural networks. Each dimension of the vector corresponds to a different feature or attribute of the content, such as its semantic meaning, syntactic role, or context in which it commonly appears. By representing content as vectors, we can perform mathematical operations on them to compare their similarity or use them as inputs to machine learning models.

## Process
<img src="https://raw.githubusercontent.com/retkowsky/Azure-Computer-Vision-in-a-day-workshop/72c07afc4fcc04a29ca19b84d3d343a09a22368e//fashionprocess.png" width=512>

## Business applications
- Digital asset management: Image retrieval can be used to manage large collections of digital images, such as in museums, archives, or online galleries. Users can search for images based on visual features and retrieve the images that match their criteria.
- Medical image retrieval: Image retrieval can be used in medical imaging to search for images based on their diagnostic features or disease patterns. This can help doctors or researchers to identify similar cases or track disease progression.
- Security and surveillance: Image retrieval can be used in security and surveillance systems to search for images based on specific features or patterns, such as in, people & object tracking, or threat detection.
- Forensic image retrieval: Image retrieval can be used in forensic investigations to search for images based on their visual content or metadata, such as in cases of cyber-crime.
- E-commerce: Image retrieval can be used in online shopping applications to search for similar products based on their features or descriptions or provide recommendations based on previous purchases.
- Fashion and design: Image retrieval can be used in fashion and design to search for images based on their visual features, such as color, pattern, or texture. This can help designers or retailers to identify similar products or trends.

## To learn more
- https://learn.microsoft.com/en-us/azure/cognitive-services/computer-vision/concept-image-retrieval
- https://learn.microsoft.com/en-us/azure/search/search-what-is-azure-search

In this notebook we took some samples fashion images are taken from this link:<br>
https://www.kaggle.com/datasets/paramaggarwal/fashion-product-images-dataset

> Note: Image retrieval is curently in public preview

## 1. Python librairies

In [1]:
import datetime
import io
import json
import os
import requests
import sys
import gradio as gr

from dotenv import load_dotenv
from PIL import Image

from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.models import Vector

from azure.storage.blob import BlobServiceClient

In [2]:
sys.version

'3.8.5 (default, Sep  4 2020, 07:30:14) \n[GCC 7.3.0]'

In [3]:
print("Today is", datetime.datetime.today())

Today is 2023-06-30 07:49:41.488610


## 2. Azure AI Services

In [4]:
load_dotenv("azure.env")

# Azure Computer Vision 4
acv_key = os.getenv("acv_key")
acv_endpoint = os.getenv("acv_endpoint")

# Azure Cognitive Search
acs_endpoint = os.getenv("acs_endpoint")
acs_key = os.getenv("acs_key")

In [5]:
# Ensure that the azure endpoints should not finished a /
if acv_endpoint.endswith("/"):
    acv_endpoint = acv_endpoint[:-1]

if acs_endpoint.endswith("/"):
    acs_endpoint = acv_endpoint[:-1]

In [6]:
# Azure storage account
blob_connection_string = os.getenv("blob_connection_string")
container_name = os.getenv("container_name")

## 3. Functions & parameters

In [7]:
# Azure Cognitive Search index name to create
index_name = "azure-fashion-demo"

# Azure Cognitive Search api version
api_version = "2023-02-01-preview"

In [8]:
def text_embedding(prompt):
    """
    Text embedding using Azure Computer Vision 4.0
    """
    version = "?api-version=" + api_version + "&modelVersion=latest"
    vec_txt_url = f"{acv_endpoint}/computervision/retrieval:vectorizeText{version}"
    headers = {"Content-type": "application/json", "Ocp-Apim-Subscription-Key": acv_key}

    payload = {"text": prompt}
    response = requests.post(vec_txt_url, json=payload, headers=headers)

    if response.status_code == 200:
        text_emb = response.json().get("vector")
        return text_emb

    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None

In [9]:
def index_stats(index_name):
    """
    Get statistics about Azure Cognitive Search index
    """
    url = (
        acs_endpoint
        + "/indexes/"
        + index_name
        + "/stats?api-version=2021-04-30-Preview"
    )
    headers = {
        "Content-Type": "application/json",
        "api-key": acs_key,
    }
    response = requests.get(url, headers=headers)
    print("Azure Cognitive Search index status for:", index_name, "\n")

    if response.status_code == 200:
        res = response.json()
        print(json.dumps(res, indent=2))

    else:
        print("Request failed with status code:", response.status_code)

In [10]:
def index_status(index_name):
    """
    Azure Cognitive Search index status
    """
    print("Azure Cognitive Search Index:", index_name, "\n")

    headers = {"Content-Type": "application/json", "api-key": acs_key}
    params = {"api-version": "2021-04-30-Preview"}
    index_status = requests.get(
        acs_endpoint + "/indexes/" + index_name, headers=headers, params=params
    )
    try:
        print(json.dumps((index_status.json()), indent=5))
    except:
        print("Request failed")

In [11]:
# Connect to Blob Storage
blob_service_client = BlobServiceClient.from_connection_string(blob_connection_string)
container_client = blob_service_client.get_container_client(container_name)
blobs = container_client.list_blobs()

first_blob = next(blobs)
blob_url = container_client.get_blob_client(first_blob).url
print(f"URL of the first blob: {blob_url}")

URL of the first blob: https://azurestorageaccountsr.blob.core.windows.net/fashionimages/fashion/0390469004.jpg


## 4. Azure Cognitive Search

In [12]:
try:
    # Setting the Azure Cognitive Search client
    print("Setting the Azure Cognitive Search client")
    search_client = SearchIndexClient(
        endpoint=acs_endpoint, credential=AzureKeyCredential(acs_key)
    )
    print("Done")
    print(search_client)

except:
    print("Request failed. Cannot create Azure Cognitive Search client:", acs_endpoint)

Setting the Azure Cognitive Search client
Done


### Azure Cognitive Search index status

In [13]:
index_stats(index_name)

Azure Cognitive Search index status for: azure-fashion-demo 

{
  "@odata.context": "https://azurecogsearcheastussr.search.windows.net/$metadata#Microsoft.Azure.Search.V2021_04_30_Preview.IndexStatistics",
  "documentCount": 10226,
  "storageSize": 155596726
}


In [14]:
index_status(index_name)

Azure Cognitive Search Index: azure-fashion-demo 

{
     "@odata.context": "https://azurecogsearcheastussr.search.windows.net/$metadata#indexes/$entity",
     "@odata.etag": "\"0x8DB7639E6632A9E\"",
     "name": "azure-fashion-demo",
     "defaultScoringProfile": null,
     "fields": [
          {
               "name": "idfile",
               "type": "Edm.String",
               "searchable": false,
               "filterable": false,
               "retrievable": true,
               "sortable": false,
               "facetable": false,
               "key": true,
               "indexAnalyzer": null,
               "searchAnalyzer": null,
               "analyzer": null,
               "normalizer": null,
               "synonymMaps": []
          },
          {
               "name": "imagefile",
               "type": "Edm.String",
               "searchable": true,
               "filterable": false,
               "retrievable": true,
               "sortable": false,
        

## 5. Webapp

In [15]:
topn = 5
imgsize = 360

footnote = "Powered by Azure Computer Vision & Azure Cognitive Search"
header_prompt = "Visual Search with Azure AI using a prompt"
header_images = "Visual Search with Azure AI using a prompt"

logo = "https://github.com/retkowsky/visual-search-azureAI/blob/main/logo.jpg?raw=true"
image = "<center> <img src= {} width=512px></center>".format(logo)

# Themes: https://huggingface.co/spaces/gradio/theme-gallery
theme = "snehilsanyal/scikit-learn"

In [16]:
def prompt_search_gradio(prompt, topn=5):
    """
    Prompt search for the gradio webapp
    """
    results_list = []
    images_list = []

    # Initialize the Azure Cognitive Search client
    search_client = SearchClient(acs_endpoint, index_name, AzureKeyCredential(acs_key))
    # Perform vector search
    vector = Vector(value=text_embedding(prompt), k=topn, fields="imagevector")
    response = search_client.search(
        search_text=prompt, vector=vector, select=["idfile", "imagefile"]
    )

    for result in response:
        image_file = result["imagefile"]
        results_list.append(image_file)

    for image_file in results_list:
        blob_client = container_client.get_blob_client(image_file)
        blob_image = blob_client.download_blob().readall()
        img = Image.open(io.BytesIO(blob_image)).resize((imgsize, imgsize))
        images_list.append(img)

    return images_list

In [17]:
prompt_examples = [
    "a red dress",
    "a red dress with long sleeves",
    "Articles with birds",
    "Hair products",
    "Ray-Ban",
    "NYC cap",
    "Camo products",
    "A blue shirt with stripped lines",
    "Ray ban with leopard frames",
    "Show me some articles with Italian names",
    "Show me some articles with Japanese on it",
    "Show me some tie-dye articles",
]

prompt = gr.components.Textbox(
    lines=2,
    label="What do you want to search?",
    placeholder="Enter your prompt for the visual search...",
)

topn_list_prompt = [""] * topn

list_img_results_image = [
    gr.components.Image(label=f"Top {i+1}: {topn_list_prompt[i]}", type="filepath")
    for i in range(topn)
]

webapp_prompt = gr.Interface(
    prompt_search_gradio,
    prompt,
    list_img_results_image,
    title=header_prompt,
    examples=prompt_examples,
    theme=theme,
    description=image,
    article=footnote,
)

In [18]:
# webapp_prompt.launch(share=True)

In [19]:
def image_embedding(imagefile):
    """
    Image embedding using Azure Computer Vision 4.0
    """
    session = requests.Session()

    version = "?api-version=" + api_version + "&modelVersion=latest"
    vec_img_url = acv_endpoint + "/computervision/retrieval:vectorizeImage" + version
    headers = {
        "Content-type": "application/octet-stream",
        "Ocp-Apim-Subscription-Key": acv_key,
    }

    try:
        with open(imagefile, "rb") as f:
            data = f.read()
        response = session.post(vec_img_url, data=data, headers=headers)
        response.raise_for_status()

        image_emb = response.json()["vector"]
        return image_emb

    except requests.exceptions.RequestException as e:
        print(f"Request Exception: {e}")
    except Exception as ex:
        print(f"Error: {ex}")

    return None

In [20]:
def image_search_gradio(imagefile, topn=5):
    """
    Image search for the gradio webapp
    """
    results_list = []
    images_list = []

    # Initialize the Azure Cognitive Search client
    search_client = SearchClient(acs_endpoint, index_name, AzureKeyCredential(acs_key))

    # Perform vector search
    response = search_client.search(
        search_text="",
        vector=Vector(value=image_embedding(imagefile), k=topn, fields="imagevector"),
        select=["idfile", "imagefile"],
    )

    for result in response:
        image_file = result["imagefile"]
        results_list.append(image_file)

    for image_file in results_list:
        blob_client = container_client.get_blob_client(image_file)
        blob_image = blob_client.download_blob().readall()
        img = Image.open(io.BytesIO(blob_image)).resize((imgsize, imgsize))
        images_list.append(img)

    return images_list

In [21]:
images_examples = [
    "images/image1.jpg",
    "images/image2.jpg",
    "images/image3.jpg",
    "images/image4.jpg",
    "images/image5.jpg",
    "images/image6.jpg",
    "images/image7.jpg",
    "images/image8.jpg",
]

refimage = gr.components.Image(label="Your image:", type="filepath", shape=((imgsize, imgsize)))
topn_list_prompt = [""] * topn

list_img_results_image = [
    gr.components.Image(label=f"Top {i+1} : {topn_list_prompt[i]}", type="filepath")
    for i in range(topn)
]

webapp_image = gr.Interface(
    image_search_gradio,
    refimage,
    list_img_results_image,
    title=header_images,
    examples=images_examples,
    theme=theme,
    description=image,
    article=footnote,
)

In [22]:
# webapp_image.launch(share=True)

### Combining the two gradio apps into one

In [23]:
visual_search_webapp = gr.TabbedInterface(
    [webapp_prompt, webapp_image],
    [
        "1) Visual search from a prompt",
        "2) Visual search from an image"
    ],
    css="body {background-color: black}",
    theme=theme,
)

/anaconda/envs/jupyter_env/lib/python3.8/site-packages/gradio/blocks.py:922: UserWarning: api_name predict already exists, using predict_1
  warnings.warn(
/anaconda/envs/jupyter_env/lib/python3.8/site-packages/gradio/blocks.py:922: UserWarning: api_name load_example already exists, using load_example_1
  warnings.warn(


In [24]:
visual_search_webapp.launch(share=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://828c9920912687a0b6.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


## 6. Post processing

We can delete the index if needed

In [25]:
index_status(index_name)

Azure Cognitive Search Index: azure-fashion-demo 

{
     "@odata.context": "https://azurecogsearcheastussr.search.windows.net/$metadata#indexes/$entity",
     "@odata.etag": "\"0x8DB7639E6632A9E\"",
     "name": "azure-fashion-demo",
     "defaultScoringProfile": null,
     "fields": [
          {
               "name": "idfile",
               "type": "Edm.String",
               "searchable": false,
               "filterable": false,
               "retrievable": true,
               "sortable": false,
               "facetable": false,
               "key": true,
               "indexAnalyzer": null,
               "searchAnalyzer": null,
               "analyzer": null,
               "normalizer": null,
               "synonymMaps": []
          },
          {
               "name": "imagefile",
               "type": "Edm.String",
               "searchable": true,
               "filterable": false,
               "retrievable": true,
               "sortable": false,
        

In [26]:
index_stats(index_name)

Azure Cognitive Search index status for: azure-fashion-demo 

{
  "@odata.context": "https://azurecogsearcheastussr.search.windows.net/$metadata#Microsoft.Azure.Search.V2021_04_30_Preview.IndexStatistics",
  "documentCount": 10226,
  "storageSize": 155596726
}


In [27]:
# delete_index(index_name)